In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

#### Getting the data

In [2]:
ticker = input("Enter the symbol you want to analyze: ")

In [3]:
ticker = yf.Ticker(ticker)

In [4]:
Balance_sheet = ticker.balance_sheet.transpose()
Income_statement = ticker.income_stmt.transpose()

#### Calculating P/E ratio
    - using diluted EPS

In [5]:
latest_price = float(ticker.history(period='1d')['Close'].values[-1])

In [6]:
ratios = latest_price / Income_statement['Diluted EPS']

In [7]:
ratios = pd.DataFrame(ratios)
ratios.rename({'Diluted EPS': 'P/E ratio'}, axis =1, inplace=True )

#### Calculating price/Book ratio